## Missing Data를 채워넣는 방법 고민

- `tavg` = ((`tmax` + `tmin`) / 2) 반올림

- `heat` = 65 - `tavg`   (난방 지수)
- `cool` = `tavg` + 65   (냉방 지수)
        - 기준 온도 화씨 65도 (18도) (참조 : https://www.wunderground.com/about/faq/degreedays.asp)


- `depart` = 아직 채워 넣을 아이디어 없음 ... (1)56%, (2)유추할 수 있는 [지난 30년의 평균 온도값] 없음
- `sunrise-sunset` = 아직 채워 넣을 아이디어 없음 (1)47%  (2)유추할 수 있는 기준 값 모르겠음


- `wetbulb 습도` = dewpoint < 습도 < wetbulb 
- `dewpoint 이슬점` = 
        - 평균온도(tavg:dry bulb), 기압(station pressure:inhg), 습도(wetbulb), 이슬점(dew point)을 대입해서 값을 구할 수 있다. 
		- 아래의 formula를 통해 시도
        - http://www.ringbell.co.uk/info/humid.htm
        - http://www.reahvac.com/tools/humidity-formulas/


- `snowfall` = codesum에 `SN SG GS PL IC UP`(눈에 관련된) 발생 시 평균값 입력
- `preciptotal` = codesumd에 `+FC TH GR UP DZ RA SH` rainfall 뿐만 아니라 melted snow라는 특성까지 포함하고 있기 때문에 전 날 or 평균값 참조


- `station pressure 기압`
- `sealevel 해면기압` = 해면기압을 구하기 위해서는 기압과 "고도"가 필요한데, `station_nbr`에 대한 위치 정보가 없으니. 잘 모르겠음

### Formula

In [2]:
# https://github.com/remcmurry/Psychropy/blob/master/psychropy.py

import math


def to_Cel(tf):
    '''
    Convert Celcius to Fahrenheit
    '''
    tc = 5/9 * (tf - 32)
    return tc


def to_Fah(tc):
    '''
    Convert Fahrenheit to Celcius
    '''
    tf = tc * (9/5) + 32
    return tf


def inhg_to_kPa(inhg):
    '''
    Calculates kilopascal (kPa)
        inhg = inches in mercury (station pressure)
        https://www.metric-conversions.org/pressure/inches-of-mercury-to-pascals.htm
    '''
    result = (inhg / 0.00029530) / 1000
    return result


def Part_press(kPa, HR):
    
    ''' 
    Function to compute partial vapor pressure in [kPa]
    From page 6.9 equation 38 in ASHRAE Fundamentals handbook (2005)
        kPa = ambient pressure [kPa]
        HR = humidity ratio [kg/kg dry air]
    '''
    result = kPa * HR / (0.62198 + HR)
    return result


def Sat_press(Tdb):

    ''' 
    Function to compute saturation vapor pressure in [kPa]
    ASHRAE Fundamentals handbood (2005) p 6.2, equation 5 and 6
        Tdb = Dry bulb temperature [degC]
        Valid from -100C to 200 C
    '''

    C1 = -5674.5359
    C2 = 6.3925247
    C3 = -0.009677843
    C4 = 0.00000062215701
    C5 = 2.0747825E-09
    C6 = -9.484024E-13
    C7 = 4.1635019
    C8 = -5800.2206
    C9 = 1.3914993
    C10 = -0.048640239
    C11 = 0.000041764768
    C12 = -0.000000014452093
    C13 = 6.5459673
 
    TK = Tdb + 273.15                     # Converts from degC to degK
    
    if TK <= 273.15:
        result = math.exp(C1/TK + C2 + C3*TK + C4*TK**2 + C5*TK**3 + 
                          C6*TK**4 + C7*math.log(TK)) / 1000
    else:
        result = math.exp(C8/TK + C9 + C10*TK + C11*TK**2 + C12*TK**3 + 
                          C13*math.log(TK)) / 1000
    return result



def Hum_rat(Tdb, Twb, kPa):
    
    ''' 
    Function to calculate humidity ratio [kg H2O/kg air]
    Given dry bulb and wet bulb temp inputs [degC]
    ASHRAE Fundamentals handbood (2005)
        Tdb = Dry bulb temperature [degC]
        Twb = Wet bulb temperature [degC]
        kPa = Ambient Pressure [kPa]
    '''

    Pws = Sat_press(Twb)
    Ws = 0.62198 * Pws / (kPa - Pws)          # Equation 23, p6.8
    if Tdb >= 0:                            # Equation 35, p6.9
        result = (((2501 - 2.326*Twb)*Ws - 1.006*(Tdb - Twb))/
                  (2501 + 1.86*Tdb - 4.186*Twb))
    else:                                   # Equation 37, p6.9
        result = (((2830 - 0.24*Twb)*Ws - 1.006*(Tdb - Twb))/
                  (2830 + 1.86*Tdb - 2.1*Twb))
    return result



def Hum_rat2(Tdb, rHR, kPa):

    ''' 
    Function to calculate humidity ratio [kg H2O/kg air]
    Given dry bulb and wet bulb temperature inputs [degC]
    ASHRAE Fundamentals handbood (2005)
        Tdb = Dry bulb temperature [degC]
        rRH = Relative Humidity [Fraction]
        P = Ambient Pressure [kPa]
    '''
    
    
    Pws = Sat_press(Tdb)
    result = 0.62198*rHR*Pws/(kPa - rHR*Pws)    # Equation 22, 24, p6.8
    return result



def Rel_hum(Tdb, Twb, kPa):

    ''' 
    Calculates relative humidity ratio
    ASHRAE Fundamentals handbood (2005)
        Tdb = Dry bulb temperature [degC]
        Twb = Wet bulb temperature [degC]
        kPa = Ambient Pressure [kPa]
    '''
      
    W = Hum_rat(Tdb, Twb, kPa)
    result = Part_press(kPa, W) / Sat_press(Tdb)   # Equation 24, p6.8
    return result



def Rel_Hum2(Tdb, Tdp):
    '''
    Calculates relative humidity ratio
        Tdb = Dry bulb temperature [degC]
        Tdp = Dew point temperature [degC]
    '''
    
    # calculate saturation vapor pressure(sav) and actual vapor pressure(avp) in millibars.
    svp = 6.11*10**(7.5*Tdb/(237.7+Tdb))
    acp = 6.11*10**(7.5*Tdp/(237.7+Tdp))
                    
    # calculate Relative Humidity Ratio
    rHR = (acp/svp)
                    
    return rHR



def Rel_Hu3(Tdb, HR, kPa):
    
    ''' 
    Calculates the relative humidity given:
        Tdb = Dry bulb temperature [degC]
        kPa = ambient pressure [kPa]
        HR = humidity ratio [kg/kg dry air]
    '''

    Pw = Part_press(kPa, HR)
    Pws = Sat_press(Tdb)
    result = Pw / Pws
    return result


def Wet_bulb(Tdb, rHR, P):
    
    ''' 
    Calculates the Wet Bulb temp given:        
        Tdb = Dry bulb temperature [degC]
        rHR = Relative humidity ratio [Fraction]
        kPa = Ambient Pressure [kPa]
    Uses Newton-Rhapson iteration to converge quickly
    '''

    # convert Fahrenheit to Celsius
    
    W_normal = Hum_rat2(Tdb, rHR, kPa)
    result = Tdb
    
    ' Solves to within 0.001% accuracy using Newton-Rhapson'    
    W_new = Hum_rat(Tdb, result, kPa)
    while abs((W_new - W_normal) / W_normal) > 0.00001:
        W_new2 = Hum_rat(Tdb, result - 0.001, kPa)
        dw_dtwb = (W_new - W_new2) / 0.001
        result = result - (W_new - W_normal) / dw_dtwb
        W_new = Hum_rat(Tdb, result, kPa)
    return result





def Dew_point(kPa, HR):

    ''' 
    Function to compute the dew point temperature (deg C)
    From page 6.9 equation 39 and 40 in ASHRAE Fundamentals handbook (2005)
        kPa = ambient pressure [kPa]
        HR = humidity ratio [kg/kg dry air]
    Valid for Dew Points less than 93 C
    '''

    C14 = 6.54
    C15 = 14.526
    C16 = 0.7389
    C17 = 0.09486
    C18 = 0.4569
    
    print(Pw)
    alpha = math.log(Pw)
    Tdp1 = C14 + C15*alpha + C16*alpha**2 + C17*alpha**3 + C18*Pw**0.1984
    Tdp2 = 6.09 + 12.608*alpha + 0.4959*alpha**2
    if Tdp1 >= 0:
        result = Tdp1
    else:
        result = Tdp2
    return result